In [ ]:
cd content


In [ ]:
!mkdir dataset

In [ ]:
cd dataset

In [ ]:
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/hamburger.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/hot%20dog.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/ice%20cream.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/pizza.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/sandwich.npy

In [ ]:
!ls

In [ ]:
cd ..

In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.utils import np_utils
from random import randint
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image


In [ ]:
# define some constants
N_FOOD = 5
FOOD = {0: "Burger", 1: "Hotdog", 2: "Icecream", 3: "Pizza" , 4: "Sandwich"}

# number of samples to take in each class
N = 5000

# some other constants
N_EPOCHS = 50

# data files in the same order as defined in FRUITS
files = ["hamburger.npy", "hot dog.npy", "ice cream.npy", "pizza.npy" , "sandwich.npy"]

In [ ]:
def load(dir, reshaped, files):
    "Load .npy or .npz files from disk and return them as numpy arrays. \
    Takes in a list of filenames and returns a list of numpy arrays."

    data = []
    for file in files:
        f = np.load(dir + file)
        if reshaped:
            new_f = []
            for i in range(len(f)):
                x = np.reshape(f[i], (28, 28))
                x = np.expand_dims(x, axis=0)
                x = np.reshape(f[i], (28, 28, 1))
                new_f.append(x)
            f = new_f
        data.append(f)
    return data


# def normalize(data):
#     "Takes a list or a list of lists and returns its normalized form"

#     return np.interp(data, [0, 255], [-1, 1])


# def denormalize(data):
#     "Takes a list or a list of lists and returns its denormalized form"

#     return np.interp(data, [-1, 1], [0, 255])


def visualize(array):
    "Visulaze a 2D array as an Image"
    array = np.reshape(array, (28,28))
    img = Image.fromarray(array)
    return img


def set_limit(arrays, n):
    "Limit elements from each array up to n elements and return a single list"
    new = []
    for array in arrays:
        i = 0
        for item in array:
            if i == n:
                break
            new.append(item)
            i += 1
    return new


def make_labels(N1, N2):
    "make labels from 0 to N1, each repeated N2 times"
    labels = []
    for i in range(N1):
        labels += [i] * N2
    return labels

In [ ]:
cd content/

In [ ]:
fod = load("dataset/", False, ['ice cream.npy'])

In [ ]:
visualize(fod[0][99])

In [ ]:
import cv2
# img=np.reshape(fod[0][99],(28,28))
cv2.imwrite('fod.jpg',img)


In [ ]:
len(fod[0][99])

In [ ]:

#second argument is True for reshaping the image to a 28x28 form. A conv net expects this format.
fod = load("dataset/", True, files)

#second argument is False because we don't need to reshape the image. An MLP net expects this format.
#fod = load("dataset/", False, files)


# limit no of samples in each class to N
fod = set_limit(fod, N)

# normalize the values
# fod = map(normalize, fod)

# define the labels
labels = make_labels(N_FOOD, N)



In [ ]:
fod=np.array(fod)
labels=np.array(labels)

In [ ]:
fod=fod/255

In [ ]:
len(fod)

In [ ]:
# prepare the data
x_train, x_test, y_train, y_test = train_test_split(fod, labels, test_size=0.05)

# one hot encoding
Y_train = np_utils.to_categorical(y_train, N_FOOD)
Y_test = np_utils.to_categorical(y_test, N_FOOD)


In [ ]:
len(y_train)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(N_FOOD, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


# train
history = model.fit(np.array(x_train), np.array(Y_train), batch_size=32, epochs=N_EPOCHS)

print ("Training complete")

print ("Evaluating model")
preds = model.predict(np.array(x_test))

score = 0
for i in range(len(preds)):
    if np.argmax(preds[i]) == y_test[i]:
        score += 1

print ("Accuracy: "), ((score + 0.0) / len(preds)) * 100


model.save("fod"+ ".h5")
print ("Model saved")


In [ ]:
loss_curve = history.history['loss']
accuracy_curve = history.history['accuracy']


In [ ]:
plt.plot(loss_curve,label='Train')
plt.legend(loc='upper left')
plt.title('Loss')
plt.show()

In [ ]:
plt.plot(accuracy_curve,label='Train')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.show()